In [6]:
dataset_dir="/workspace/data/thermix_data/thermset_v1.1"
group="Fiona_001"
day=2
start_hour=12

#modelname="2016-11-04_145241"
#checkpoint=6097
modelname="2016-11-11_073507"
checkpoint=2622
classes_names=["lying","-"]
classes_count=len(classes_names)

modelpath="/workspace/data/thermix_data/tf_training/%s/checkpoint-%d"%(modelname,checkpoint)
tf_workspace ="/workspace/data/thermix_data/tf_evaluation"

fps=30

In [7]:
import os

input_filepath=os.path.join(tf_workspace,"%s_%s_day%d.in"%(group, modelname, day))
output_filepath=os.path.join(tf_workspace,"%s_%s_day%d.out"%(group, modelname, day))

video_filepath=os.path.join(tf_workspace,"%s_%s_day%d.mov"%(group, modelname, day))

In [8]:
from datetime import datetime, timedelta

annotations_filename = "%s_annotations.txt"%group
annotations_filepath = os.path.join(dataset_dir, group, annotations_filename)

clips_path = os.path.join(dataset_dir,group)
clips = [(c, datetime.strptime(c,"%Y-%m-%d_%H.%M.%S")) for c in os.listdir(clips_path) if c != annotations_filename]

clips = sorted(clips, key=lambda c:c[1])

start_date=(clips[0][1]+timedelta(days=day-1)).replace(hour=start_hour, minute=0, second=0)
end_date=start_date+timedelta(days=1)
print start_date, end_date 

clips_to_process=[]
for c,d in clips:
    if start_date <= d <= end_date:
        clips_to_process.append(c)
      
input_dir = os.path.dirname(input_filepath)
if not os.path.isdir(input_dir):
    os.makedirs(input_dir)
  
frames_to_process = []
for c in clips_to_process:
    c_path = os.path.join(clips_path, c)
    frames=sorted(os.listdir(c_path), key=lambda x:int(x[:-4]))
    for fr in frames:
        fr_path=os.path.join(c_path, fr)
        frames_to_process.append(fr_path)
            
with open(input_filepath, "w") as f:
    for fr_path in frames_to_process:
        f.write("%s 0\n"%fr_path)

print "frames to predict: %d"%len(frames_to_process)
print frames_to_process[:100]

2016-07-09 12:00:00 2016-07-10 12:00:00
frames to predict: 169968
['/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/0.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/1.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/2.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/3.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/4.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/5.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/6.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/7.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/8.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/9.png', '/workspace/data/thermix_data/thermset_v1.1/Fiona_001/2016-07-09_12.00.00/10.png', '/workspace/data/thermix_data/

In [10]:
import tflearn, tensorflow as tf
from tflearn.layers.core import input_data, dropout, fully_connected,reshape
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import batch_normalization#local_response_normalization
#from tflearn.layers.estimator import regression
from training_utils import thermal_image_preloader

def model(input_placeholder=None, category_weights=[1.,1.]):
    
    cat_weights_c = tf.constant(category_weights, tf.float32)
    
    tf_data = input_placeholder or tf.placeholder(tf.float32, shape=(None, 224, 224))
    network = input_data(placeholder=tf_data)
    
    network = reshape(network, [-1,224,224,1])
    
    network = batch_normalization(network)
    
    network = conv_2d(network, 96, 7, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = batch_normalization(network)
    
    network = conv_2d(network, 384, 3, activation='relu')
    network = batch_normalization(network)
    
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    network = fully_connected(network, 2, activation='linear')
    network = tf.mul(network, cat_weights_c)
    network = tflearn.activations.softmax (network)
    
    return network, tf_data

graph = tf.Graph()
with graph.as_default():
    # Model variables
    #The network to be used
    with tf.device('/gpu:0'):
        net, X_ph = model(category_weights=[2.76009096, 0.61061502])

        init = tf.initialize_all_variables()
        vars_to_restore = []
        for v in tf.all_variables():
            if "is_training" in v.name:
                print v.name
                continue
            else:
                print v.name, 'restored'
            vars_to_restore.append(v)
        saver = tf.train.Saver(vars_to_restore)

BatchNormalization/beta:0 restored
BatchNormalization/gamma:0 restored
BatchNormalization/moving_mean:0 restored
BatchNormalization/moving_variance:0 restored
BatchNormalization/is_training:0
Conv2D/W:0 restored
Conv2D/b:0 restored
BatchNormalization_1/beta:0 restored
BatchNormalization_1/gamma:0 restored
BatchNormalization_1/moving_mean:0 restored
BatchNormalization_1/moving_variance:0 restored
Conv2D_1/W:0 restored
Conv2D_1/b:0 restored
BatchNormalization_2/beta:0 restored
BatchNormalization_2/gamma:0 restored
BatchNormalization_2/moving_mean:0 restored
BatchNormalization_2/moving_variance:0 restored
Conv2D_2/W:0 restored
Conv2D_2/b:0 restored
BatchNormalization_3/beta:0 restored
BatchNormalization_3/gamma:0 restored
BatchNormalization_3/moving_mean:0 restored
BatchNormalization_3/moving_variance:0 restored
Conv2D_3/W:0 restored
Conv2D_3/b:0 restored
BatchNormalization_4/beta:0 restored
BatchNormalization_4/gamma:0 restored
BatchNormalization_4/moving_mean:0 restored
BatchNormalizati

In [33]:
X, _ =thermal_image_preloader(input_filepath, (224,224), mode='file', normalize=False,
                              grayscale=True, categorical_labels=True,
                              files_extension=None, filter_channel=False)

# Launch the graph
with tf.Session(graph=graph, config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True)) as sess:
    tflearn.config.is_training(False,sess)
    sess.run(init)
    saver.restore(sess,modelpath)
    
    prediction = []
    i=0
    step = 500
    while len(prediction) <len(X):
        res = net.eval({X_ph:X[i:i+step]})
        for r in res:
            sorted_indexes = sorted(range(classes_count), key=lambda x:r[x], reverse=True)
            prediction.append((sorted_indexes[0], r[sorted_indexes[0]]))
        i+=step

BatchNormalization/is_training:0


In [34]:
#save results
with open(output_filepath, "w") as f:
    for i in range(len(frames_to_process)):
        fr_path = frames_to_process[i]
        label = prediction[i][0]
        score = prediction[i][1]
        f.write("%s %d:%f\n"%(fr_path,label,score))

In [ ]:
from datetime import datetime, timedelta

def str_date(date):
    return date.strftime("%Y-%m-%d %H:%M:%S")

with open(output_filepath) as f:
    lines=f.readlines()

video_lens = {}
for line in lines:
    fr,info = line.split()
    
    video_name= os.path.basename(os.path.dirname(fr))
    if not video_lens.get(video_name):
        video_lens[video_name]= 1
    else:
        video_lens[video_name]+= 1
    
category_lines = {}
cat_by_index = classes_names
last_cat=None
for line in lines:
    fr,info = line.split()
    cat,score=info.split(":")
    str_cat=cat_by_index[int(cat)]
    
    video_name= os.path.basename(os.path.dirname(fr))
    frame_index=int(os.path.basename(fr)[:-4])
    
    start_date= str_date(datetime.strptime(video_name,"%Y-%m-%d_%H.%M.%S")+timedelta(seconds=(float(frame_index)/video_lens[video_name])*5))
    
    if not category_lines.get(str_cat):
        category_lines[str_cat]=[]
    
    if not last_cat:
        category_lines[str_cat].append(start_date)
        last_cat = str_cat
        continue
        
    if last_cat != str_cat:
        category_lines[last_cat].extend([start_date,None])
        category_lines[str_cat].append(start_date)
        last_cat = str_cat
        
    if line == lines[-1]:
        category_lines[str_cat].append(start_date)

for c in category_lines.keys():
    if category_lines[c][-1] is None:
        category_lines[c].pop(-1)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
cat_order = ["-", "lying"]
for cat in cat_order:
    trace_p = go.Scatter(
        x = category_lines[cat],
        y = [cat for _ in category_lines[cat]],
        mode = 'lines',
        name = '%s' %(cat)
    )
    
    data.append(trace_p)

layout = dict(title='%s Day %d timeline'%(group, day))
fig = dict(data=data, layout=layout)
    
py.iplot(fig, filename='%s Day %d timeline'%(group, day))

In [35]:
from draw_utils import draw_classification_on_image
import shutil

temp_dir= os.path.join(tf_workspace, "temp")
if os.path.isdir(temp_dir):
    shutil.rmtree(temp_dir)

os.makedirs(temp_dir)

labels=classes_names
for i in range(len(frames_to_process)):
    
    fr = frames_to_process[i]
    score = prediction[i][1]
    label = labels[prediction[i][0]]
    draw_classification_on_image(fr, os.path.join(temp_dir, "%020d.png"%i), score, label, stretch_image=True, add_date=True)

In [6]:
if os.path.exists(video_filepath):
    os.remove(video_filepath)
    
os.system("ffmpeg -pattern_type glob -i '%s/*.png' -r %d -c:v libx264 %s" % (temp_dir, fps, video_filepath))
#shutil.rmtree(temp_dir)
os.system("cd youtube_upload && python upload_video.py --file=%s --title='%s Day %d' && cd .."% (video_filepath, group, day))

0